<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li></ul></div>

# Защита персональных данных клиентов

Нам нужно защитить данные клиентов страховой компании «Хоть потоп». Для этого разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуем корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

## Знакомство с данными

In [ ]:
data = pd.read_csv('/datasets/insurance.csv')

In [ ]:
display(data.head())
display(data.info())
display(data.describe())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

При шифровании данных качество модели не изменится.

**Обоснование:**

Признаки умножили на обратимую матрицу :

$$w_1 = ((XP)^T XP)^{-1} (XP)^T y$$
    
Раскрыли скобки.
Транспонированное произведение матриц равно произведению транспонированных матриц, взятых в обратном порядке $(А · B)^T = B^T · А^T$:

$$w_1 = (P^T X^T XP)^{-1} P^T X^T $$

Вынесли за скобки $P^T и P{-1}$, использую формулу $(AB)^{-1} = B^{-1} A^{-1} y$:


$$w_1 = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y$$

Собрали единичную матрицу из $(P^T)^{-1} P^T$. (Умножение матрицы на обратную матрицу равно единичной матрице):

$$w_1 = P^{-1} (X^T X)^{-1} X^T y$$

Подставили в формулу предсказания $a = Xw$ и получили:

$$a_1 = XPw_1$$

Подставили $w_1$:

$$a_1 = XP  P^{-1} (X^T X)^{-1} X^T y$$

Снова собрали единичную матрицу, на этот раз из $P  P^{-1}$ и получили объединенную формулу предсказания и обучения:

$$
a_1 = X (X^T X)^{-1} X^T y
$$

$$
a = Xw
$$

$$
w = (X^T X)^{-1} X^T y
$$

**Вывод:** в результате преобразований подтвердили, что умножение признаков на обратимую матрицу не изменит качество модели.

## Алгоритм преобразования

**Алгоритм**

Для защиты информации умножили матрицу признаков на марицу шифрования.
За марицу шифрования взяли обратимую матрицу, сгенерированную случайным образом.

**Обоснование**

Матрица шифрования должна иметь необходимую размерность, где n - количество признаков для регрессии. Таким образом итоговая матрица будет иметь туже размерность, что и исходная матрица. Обратная матрица существует только для квадратных матриц, а значит необходимый размер матрицы шифрования 4х4.

## Проверка алгоритма

Разделили данные на признаки и целевой признак. Создали случайную квадратную матрицу с необходимой размерностью. Проверили матрицу гифрования на обратимость:

In [ ]:
X = data.drop('Страховые выплаты', axis=1)
y = data['Страховые выплаты']

In [ ]:
encryp_matrix = np.random.normal(size=(X.shape[1], X.shape[1]))
encryp_matrix

array([[-1.33617072,  0.31933813,  1.1984398 , -0.51733166],
       [ 1.94971229, -0.87520599,  0.22942271, -0.08180798],
       [-0.58499576,  0.35501441,  0.3387533 , -0.59928042],
       [-0.41177737, -0.44898427, -0.90715829, -0.24996258]])

In [ ]:
encryp_matrix_inv = np.linalg.inv(encryp_matrix)
encryp_matrix_inv_inv = np.linalg.inv(encryp_matrix_inv)
encryp_matrix_inv_inv

array([[-1.33617072,  0.31933813,  1.1984398 , -0.51733166],
       [ 1.94971229, -0.87520599,  0.22942271, -0.08180798],
       [-0.58499576,  0.35501441,  0.3387533 , -0.59928042],
       [-0.41177737, -0.44898427, -0.90715829, -0.24996258]])

Зашифровади даннные, умножив исходную матрицу на матрицу шифрования:

In [ ]:
X_encryp = X @ encryp_matrix
X_encryp

,0,1,2,3
0,-28937.599632,17572.701416,16811.861096,-29728.430394
1,-22140.564039,13449.838945,12882.271536,-22776.669197
2,-12228.369385,7429.921539,7120.472476,-12587.261311
3,-24354.202950,14784.823411,14129.016005,-24992.211525
4,-15215.133664,9241.689551,8849.083302,-15644.026991
...,...,...,...,...
4995,-20830.580381,12648.610536,12098.102188,-21397.101644
4996,-30587.899586,18572.548855,17757.565927,-31405.325590
4997,-19793.185704,12016.586254,11486.510873,-20317.742419
4998,-19089.039311,11588.688910,11080.757045,-19599.536822


Построили модель регрессии на нешированных данных и на шированных данных для сравнения:


In [ ]:
orig_model = LinearRegression()
orig_model.fit(X, y)
orig_predictions = orig_model.predict(X)
print('R2 для нешифрованной матрицы:', r2_score(y,orig_predictions))

encryp_model = LinearRegression()
encryp_model.fit(X_encryp, y)
encryp_predictions = encryp_model.predict(X_encryp)
print('R2 для шифрованной матрицы:', r2_score(y,encryp_predictions))

R2 для нешифрованной матрицы: 0.4249455028666801
R2 для шифрованной матрицы: 0.42494550286667854


**Выводы:**
- Данные были загружены и изучены.
- Был разработан алгоритм шифрования данных с использование случайно сгенерированной матрицы подходящей размерности.
- В результате шифрования данные стали в достаточной мере нечитаемыми, при этом размерность сохранилась.
- При проверке работы модели регресси на нешированных данных и на шированных данных существенных различий в показателе R2 не оказалось, следовательно шифровка данных не повлияла на работы модели.